In [1]:
import os
import json
import pandas as pd
import traceback

In [21]:
from langchain.llms import GooglePalm
from dotenv import load_dotenv

load_dotenv() 

True

In [23]:
KEY=os.getenv("PALM_API_KEY")

In [25]:
llm = GooglePalm(google_api_key=KEY, temperature=0.1)

In [26]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [27]:
llm

GooglePalm(client=<module 'google.generativeai' from 'c:\\Users\\Sheela Sai kumar\\Documents\\GenAI\\mcqgen\\env\\lib\\site-packages\\google\\generativeai\\__init__.py'>, google_api_key=SecretStr('**********'), temperature=0.1)

In [29]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [30]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [31]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [32]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [33]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [34]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [35]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [36]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [38]:
file_path= r"C:\Users\Sheela Sai kumar\Documents\GenAI\mcqgen\data.txt"

In [39]:
file_path

'C:\\Users\\Sheela Sai kumar\\Documents\\GenAI\\mcqgen\\data.txt'

In [40]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [41]:
print(TEXT)

Deep learning is the subset of machine learning methods based on artificial neural networks (ANNs) with representation learning. The adjective "deep" refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.[2]

Deep-learning architectures such as deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.[3][4][5]

Artificial neural networks were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains. Specifically, artificial neural net

In [42]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [43]:
NUMBER=5 
SUBJECT="deep learning"
TONE="advance"

In [44]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

c:\Users\Sheela Sai kumar\Documents\GenAI\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Deep learning is the subset of machine learning methods based on artificial neural networks (ANNs) with representation learning. The adjective "deep" refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.[2]

Deep-learning architectures such as deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.[3][4][5]

Artificial neural networks were inspired by information processing and distributed communication nodes in b

In [54]:
response

{'text': 'Deep learning is the subset of machine learning methods based on artificial neural networks (ANNs) with representation learning. The adjective "deep" refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.[2]\n\nDeep-learning architectures such as deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.[3][4][5]\n\nArtificial neural networks were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains. Specifically, artific

In [59]:
quiz=response["quiz"]

In [60]:
print(quiz)

```json
{
  "1": {
    "mcq": "What is the subset of machine learning methods based on artificial neural networks with representation learning?",
    "options": {
      "a": "Deep learning",
      "b": "Supervised learning",
      "c": "Semi-supervised learning",
      "d": "Unsupervised learning"
    },
    "correct": "a"
  },
  "2": {
    "mcq": "Which of the following is not a deep learning architecture: deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks or transformers?",
    "options": {
      "a": "deep neural networks",
      "b": "deep belief networks",
      "c": "recurrent neural networks",
      "d": "convolutional neural networks",
      "e": "transformers"
    },
    "correct": "e"
  },
  "3": {
    "mcq": "What are the differences between artificial neural networks and biological brains?",
    "options": {
      "a": "Artificial neural networks tend to be static and symbolic, while the biological brain of most living organ

In [74]:
quiz2 = quiz.strip("```json")

In [76]:
quiz_json = json.loads(quiz2)


In [78]:
print(quiz_json)

{'1': {'mcq': 'What is the subset of machine learning methods based on artificial neural networks with representation learning?', 'options': {'a': 'Deep learning', 'b': 'Supervised learning', 'c': 'Semi-supervised learning', 'd': 'Unsupervised learning'}, 'correct': 'a'}, '2': {'mcq': 'Which of the following is not a deep learning architecture: deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks or transformers?', 'options': {'a': 'deep neural networks', 'b': 'deep belief networks', 'c': 'recurrent neural networks', 'd': 'convolutional neural networks', 'e': 'transformers'}, 'correct': 'e'}, '3': {'mcq': 'What are the differences between artificial neural networks and biological brains?', 'options': {'a': 'Artificial neural networks tend to be static and symbolic, while the biological brain of most living organisms is dynamic (plastic) and analog', 'b': 'Artificial neural networks tend to be dynamic (plastic) and analog, while the biolog

In [79]:
quiz_table_data = []
for key, value in quiz_json.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [80]:
quiz_table_data

[{'MCQ': 'What is the subset of machine learning methods based on artificial neural networks with representation learning?',
  'Choices': 'a: Deep learning | b: Supervised learning | c: Semi-supervised learning | d: Unsupervised learning',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is not a deep learning architecture: deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks or transformers?',
  'Choices': 'a: deep neural networks | b: deep belief networks | c: recurrent neural networks | d: convolutional neural networks | e: transformers',
  'Correct': 'e'},
 {'MCQ': 'What are the differences between artificial neural networks and biological brains?',
  'Choices': 'a: Artificial neural networks tend to be static and symbolic, while the biological brain of most living organisms is dynamic (plastic) and analog | b: Artificial neural networks tend to be dynamic (plastic) and analog, while the biological brain of most living organisms is

In [81]:
quiz=pd.DataFrame(quiz_table_data)

In [82]:
quiz.head()

,MCQ,Choices,Correct
0,What is the subset of machine learning methods...,a: Deep learning | b: Supervised learning | c:...,a
1,Which of the following is not a deep learning ...,a: deep neural networks | b: deep belief netwo...,e
2,What are the differences between artificial ne...,a: Artificial neural networks tend to be stati...,a
3,Which of the following fields has not been app...,a: computer vision | b: speech recognition | c...,j
4,Which of the following is not a low quality mo...,a: artificial neural networks | b: biological ...,a


In [84]:
quiz.to_csv("deeplearning.csv",index=False)